### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment (example)

In [ ]:
slice_name="mp-" + fablib.get_bastion_username()

node_conf = [
 {'name': "client",  'cores': 4, 'ram': 8, 'disk': 30, 'image': 'default_ubuntu_20', 'packages': ["moreutils"]}, 
 {'name': "router",  'cores': 4, 'ram': 8, 'disk': 30, 'image': 'default_ubuntu_20', 'packages': ["moreutils"]}, 
 {'name': "aggr",    'cores': 4, 'ram': 8, 'disk': 30, 'image': 'default_ubuntu_20', 'packages': ["moreutils"]}, 
 {'name': "server",  'cores': 4, 'ram': 8, 'disk': 30, 'image': 'default_ubuntu_20', 'packages': ["moreutils"]}
]
net_conf = [
 {"name": "net0", "subnet": "10.10.0.0/24", "nodes": [{"name": "client",  "addr": "10.10.0.100"}, {"name": "router",  "addr": "10.10.0.1"}]},
 {"name": "net1", "subnet": "10.10.1.0/23", "nodes": [{"name": "router",  "addr": "10.10.1.1"},   {"name": "aggr",    "addr": "10.10.1.2"}]},
 {"name": "net2", "subnet": "10.10.2.0/23", "nodes": [{"name": "router",  "addr": "10.10.2.1"},   {"name": "aggr",    "addr": "10.10.2.2"}]},
 {"name": "net3", "subnet": "10.10.3.0/24", "nodes": [{"name": "server",  "addr": "10.10.3.100"},  {"name": "aggr",   "addr": "10.10.3.1"}]}
]

route_conf = [
 {"addr": "10.10.3.0/24", "gw": "10.10.0.1", "nodes": ["client"]},
 {"addr": "10.10.0.0/24", "gw": "10.10.3.1", "nodes": ["server"]}

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Configure queues

In [ ]:
# all of the experiment factors. For now, just one value. Later, we make these lists
# and we can systematically run a full factorial experiment.
exp = {
    'link_rate':  130,  # rate of interface at endpoints and router reverse path, in Mbps.
    'link_buf':   2.6,  # size of interface at endpoints and router reverse path, in Mbit.
    'link_delay': 10,   # in ms, applied at aggr only
    'p1_rate':    130,  # rate of router egress interface for path 1, in Mbps 
    'p1_buf':     2.6,  # size of router egress buffer for path 1, in Mbit
    'p1_delay':   0.01, # delay of router egress interface for path 1, in ms 
    'p2_rate':    130,  # rate of router egress interface for path 2, in Mbps 
    'p2_buf':     2.6,  # size of router egress buffer for path 2, in Mbit
    'p2_delay':   0.01, # delay of router egress interface for path 2, in ms 
    'p2_prob':    0.5,  # probability of being marked for path 2 
    'lb_type':    "packet", # can be "packet" or "flow"
    'trial':      1     # trial index
}

In [ ]:
client_node = slice.get_node(name="client")
client_iface  = client_node.get_interface(network_name = "net0")
client_ifname = client_iface.get_device_name()

client_tc_cmd = '''
    sudo tc qdisc del dev {iface} root  
    sudo tc qdisc add dev {iface} root handle 1: htb default 3  
    sudo tc class add dev {iface} parent 1:2 classid 1:3 htb rate {rate}Mbit  
    sudo tc qdisc add dev {iface} parent 1:3 bfifo limit  {buf}mbit 
    '''.format(iface=client_ifname, rate=exp['link_rate'], buf=exp['link_buf'])

client_node.execute(client_tc_cmd)


In [ ]:
router_node = slice.get_node(name="router")
router_iface_0  = router_node.get_interface(network_name = "net0")
router_iface_1  = router_node.get_interface(network_name = "net1")
router_iface_2  = router_node.get_interface(network_name = "net2")
router_ifname_0 = router_iface_0.get_device_name()
router_ifname_1 = router_iface_1.get_device_name()
router_ifname_2 = router_iface_2.get_device_name()

router_tc_cmd = '''

    # configure next hop for "top" path
    sudo ip route add 10.10.3.0/24 via 10.10.1.2 dev {iface1}

    sudo tc qdisc del dev {iface1} root  
    sudo tc qdisc add dev {iface1} root handle 1: htb default 3  
    sudo tc class add dev {iface1} parent 1:2 classid 1:3 htb rate {r1}Mbit quantum 1514
    sudo tc qdisc add dev {iface1} parent 1:3 handle 3: netem delay {d1}ms
    sudo tc qdisc add dev {iface1} parent 3: bfifo limit {b1}mbit 


    # configure next hop for "bottom" path
    sudo ip route add 10.10.3.0/24 via 10.10.2.2 dev {iface2} table 100
    sudo ip rule add fwmark 3 table 100 

    sudo tc qdisc del dev {iface2} root  
    sudo tc qdisc add dev {iface2} root handle 1: htb default 3  
    sudo tc class add dev {iface2} parent 1:2 classid 1:3 htb rate {r2}Mbit quantum 1514 
    sudo tc qdisc add dev {iface2} parent 1:3 handle 3: netem delay {d2}ms
    sudo tc qdisc add dev {iface2} parent 3: bfifo limit {b2}mbit 


    # configure reverse path
    sudo tc qdisc del dev {iface0} root  
    sudo tc qdisc add dev {iface0} root handle 1: htb default 3  
    sudo tc class add dev {iface0}parent 1:2 classid 1:3 htb rate {rl}Mbit  
    sudo tc qdisc add dev {iface0} parent 1:3 bfifo limit  {bl}mbit 

    '''.format(iface0=router_ifname_0, iface1=router_ifname_1, iface2=router_ifname_2, 
            r1=exp['p1_rate'],   b1=exp['p1_buf'], d1=exp['p1_delay'], 
            r2=exp['p2_rate'],   b2=exp['p2_buf'], d2=exp['p2_delay'],
            rl=exp['link_rate'], bl=exp['link_buf'])
    
router_node.execute(router_tc_cmd)

In [ ]:
if exp['lb_type']=="packet":
    router_iptables_cmd = '''
        # flush first!
        sudo iptables -t mangle -F
        sudo iptables -A PREROUTING -m statistic --mode nth --every {n_packets} --packet 0 -t mangle --destination 10.10.3.100/24 --source 10.10.0.100/1 -j MARK --set-mark 3
        sudo iptables -A PREROUTING -m mark --mark 3 -t mangle -j RETURN
    '''

    
if exp['lb_type']=="flow":

    router_iptables_cmd = '''
        # flush first!
        sudo iptables -t mangle -F
        # Mark all flows to 2
        sudo iptables -t mangle -A PREROUTING -i {iface0} -m conntrack --ctstate NEW -j CONNMARK --set-mark 2
        # Mark half of the flows to 1
        sudo iptables -t mangle -A PREROUTING -i {iface0} -m conntrack --ctstate NEW -m statistic --mode random --probability {prob} -j CONNMARK --set-mark 1
        # restore and overwrite (?) the mark of previously marked flows
        sudo iptables -t mangle -A PREROUTING -i {iface0} -m conntrack --ctstate ESTABLISHED,RELATED -j CONNMARK --restore-mark

        # Mark packets 3 if they belong to flows marked 1
        sudo iptables -t mangle -A PREROUTING -m connmark --mark 1 -j MARK --set-mark 3
        # if a packet is marked with 3 return
        sudo iptables -A PREROUTING -m mark --mark 3 -t mangle -j RETURN

        # not used in this example - you can mark more packets with different marks as follows 
        #sudo iptables -t mangle -A PREROUTING -m connmark --mark 2 -j MARK --set-mark 4
        #sudo iptables -A PREROUTING -m mark --mark 4 -t mangle -j RETURN
    '''.format(iface0=router_ifname_0, prob=exp['p2_prob'], n_packets=int(1/exp['p2_prob']))

    
router_node.execute(router_iptables_cmd)

In [ ]:
router_node.execute("sudo iptables -L -n -t mangle")

In [ ]:
aggr_node = slice.get_node(name="aggr")
aggr_iface_3  = aggr_node.get_interface(network_name = "net3")
aggr_iface_1  = aggr_node.get_interface(network_name = "net1")
aggr_iface_2  = aggr_node.get_interface(network_name = "net2")
aggr_ifname_3 = aggr_iface_3.get_device_name()
aggr_ifname_1 = aggr_iface_1.get_device_name()
aggr_ifname_2 = aggr_iface_2.get_device_name()

aggr_tc_cmd = '''

    # configure next hop for "top" path
    sudo ip route add 10.10.0.0/24 via 10.10.1.1 dev {iface1}

    sudo tc qdisc del dev {iface1} root  
    sudo tc qdisc add dev {iface1} root handle 1: htb default 3  
    sudo tc class add dev {iface1} parent 1:2 classid 1:3 htb rate {r1}Mbit quantum 1514
    sudo tc qdisc add dev {iface1} parent 1:3 bfifo limit {b1}mbit  


    # configure next hop for "bottom" path
    sudo ip route add 10.10.0.0/24 via 10.10.2.1 dev {iface2} table 100
    sudo ip rule add fwmark 3 table 100 

    sudo tc qdisc del dev {iface2} root  
    sudo tc qdisc add dev {iface2} root handle 1: htb default 3  
    sudo tc class add dev {iface2} parent 1:2 classid 1:3 htb rate {r2}Mbit quantum 1514 
    sudo tc qdisc add dev {iface2} parent 1:3 handle 3: netem delay {d2}ms
    sudo tc qdisc add dev {iface2} parent 1:3 bfifo limit {b2}mbit  


    # configure path to server

    sudo tc qdisc del dev {iface3} root  
    sudo tc qdisc add dev {iface3} root handle 1: htb default 3  
    sudo tc class add dev {iface3} parent 1:2 classid 1:3 htb rate {rl}Mbit quantum 1514 
    sudo tc qdisc add dev {iface3} parent 1:3 handle 3: netem delay {dl}ms
    sudo tc qdisc add dev {iface3} parent 3: bfifo limit {bl}mbit 
    
    
    '''.format(iface3=aggr_ifname_3, iface1=aggr_ifname_1, iface2=aggr_ifname_2, 
            r1=exp['p1_rate'],   b1=exp['p1_buf'],   d1=exp['p1_delay'], 
            r2=exp['p2_rate'],   b2=exp['p2_buf'],   d2=exp['p2_delay'],
            rl=exp['link_rate'], bl=exp['link_buf'], dl=exp['link_delay'])
            
aggr_node.execute(aggr_tc_cmd)

In [ ]:
server_node = slice.get_node(name="server")
server_iface  = server_node.get_interface(network_name = "net3")
server_ifname = server_iface.get_device_name()

server_tc_cmd = '''
    sudo tc qdisc del dev {iface} root  
    sudo tc qdisc add dev {iface} root handle 1: htb default 3  
    sudo tc class add dev {iface} parent 1:2 classid 1:3 htb rate {rate}Mbit  
    sudo tc qdisc add dev {iface} parent 1:3 bfifo limit  {buf}mbit 
    '''.format(iface=server_ifname, rate=exp['lr'], buf=exp['lb'])


server_node.execute(server_tc_cmd)